### Load Data

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score

from tqdm import tqdm
tqdm.pandas(desc='progress-bar')
from sklearn import utils
from gensim.models import Doc2Vec as d2v
import gensim
from gensim.models.doc2vec import TaggedDocument
import pprint
from imblearn.over_sampling import SMOTE

import pickle
from pathlib import Path

C:\Users\cagli\Anaconda3\envs\dashmote\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
connection = sqlite3.connect('messages_info.db')
query = 'SELECT * FROM data'

data = pd.read_sql(query, connection)

### Preprocessing

In [3]:
def clean_string(string):
    delete_punctuation = re.compile('[^a-zA-Z0-9]')
    clean_string = re.sub(delete_punctuation, ' ', string)
    return clean_string

In [4]:
def label_messages(column, label_type):
    labeled = []
    for i, cell in enumerate(column):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(cell.split(), [label]))
    return labeled

In [5]:
data.message = data.message.map(clean_string)

X_train, X_test, y_train, y_test = train_test_split(data.message, data.loc[:, 'related':], train_size=0.75,
                                                    random_state=42)

X_train = label_messages(X_train, 'train')
X_test = label_messages(X_test, 'test')

In [6]:
drop_list = []

for col in y_train.columns:
    y_train[col] = pd.to_numeric(y_train[col], errors='raise')
    if y_train[col].sum() == 0:
        drop_list.append(col)

for col in y_test.columns:
    y_test[col] = pd.to_numeric(y_test[col], errors='raise')

y_test.drop(drop_list, inplace=True, axis=1)
y_train.drop(drop_list, inplace=True, axis=1)

In [7]:
# Parameters for Doc2Vec
dm=0
vector_size = 300
negative=5
min_count=1
alpha=0.065
epochs = 30

model_dbow = d2v(dm=dm, vector_size=vector_size, negative=negative, min_count=min_count, alpha=alpha, min_alpha=alpha)
model_dbow.build_vocab([x for x in tqdm (X_train + X_test)])

for epoch in range(epochs):
    model_dbow.train(utils.shuffle([x for x in tqdm(X_train + X_test)]), total_examples=len(X_train + X_test), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26216/26216 [00:00<00:00, 4381664.62it/s]


### Preparing Data

In [8]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [9]:
train_vectors = get_vectors(model_dbow, len(X_train), 300, 'train')
test_vectors = get_vectors(model_dbow, len(X_test), 300, 'test')

### Machine Learning

In [ ]:
class ModelStack():
    def __init__(self, ):